# Graph Databases for Device Management


## Introduction
Most of us are very familiar with the structure of relational databases and how to model data for it. However schemas developed for relational databases tend to become complicated and resistant to change. In addition as more tables are added to a schema the joins required for both reads and writes become more complicated which slows down database operations.

One solution to this problem is the use of document store databases, such as DynamoDB. These databases tend to be schemaless making it easier to change the data stored as requirements evolve. In addition a common principal of many of these databases is that you should be able to get all the information you require in a single read operation which often requires you to carefully consider the indexes you create and denormalise your data to model relationships. While denormalising data does make reading easier it does create difficulty in ensuring consistency during write operations.

In general relational databases, as their names suggest, are good at modelling the relationships between different pieces of data and performing adhoc queries although the complexity of these relationship increases the length of time these operation take. Document store databases on the other hand tend to be very fast to find individual documents but the modelling of relationship is an acquired skill that becomes increasingly complicated as the types of relationships increase.

This all struck me as a little like the Goldilocks principal, one is too hot and the other too cold, so where is the "just right" solution? So I decide to have a look at graph databases to see if they might be a better solution.


## Context
For the purposes of this discussion we will be considering how to implement a database to hold data about computing devices that are located at a particular location and can be owned and managed by different organisations.

It should be stated from the outset that this is not really a complicated or large domain, in fact you could probably store all the data in a csv file and still get acceptable performance, but it will allow us to identify some of the features of graph databases that are useful.

### Requirements
As always with development knowing the requirements we are trying to meet is essential. Listed below are some of the requirements that I have identified for this domain.

1. Be able to find an individual device.
2. Be able to find all devices at a given location.
3. Find all devices for the sites associated with an organisation.
4. Be able to find all devices with a particular status.
5. Find all devices of a particular type.

## The Database
For the examples in this notebook we will be using Neo4j. You can download the Neo4j desktop application from their [website](https://neo4j.com/download-center/). This application will allow you to create a local database to try things out. There is a AWS graph database service available, called Neptune, but as there doesn't seem to be any free tier available on the service it was decided not to use that at this stage. All code examples in this document are in Python. However Neo4j itself is a Java application and client libraries are available in a number of different languages, including Javascript.

Once you have a Neo4j installed you can start a local database and connect to it as shown below.

In [1]:
from neo4j import GraphDatabase, Node
from pandas import DataFrame, read_csv
from scripts.vis import draw

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"), encrypted=False)

## Requirement - Find an Individual Device

Assuming we are starting with a blank database the first order of business is to put some device data into the graph. Like DynamoDB and other Document Store databases the nodes in a Neo4j database as schemaless, that is the information stored in two different nodes that are labelled as a device does not need to be the same.

This make things easy for us when we are starting as we don't need to try and capture all the information we require at the outset, rather we can put in the essential information now and easily add further information at a later stage.

In [2]:
# Create a device and print the newly created node out.
device = {
    'id': '000000000000',
    'type': 'IntelPC',
    'name': 'Device 1A'
}

def create_device(tx, device):
    result = tx.run("CREATE (a:Device) "
                    "SET a = $device "
                    "RETURN a", device=device)
    return result.single()

with driver.session() as session:
    device = session.write_transaction(create_device, device)
    dataFrame = DataFrame.from_records(device)
    display(dataFrame)

,id,name,type
0,000000000000,Device 1A,IntelPC


Woooh... we have created our first node in the database. Obviously we are going to need to add a few more to make things more interesting but first it is worth stopping to consider the access patterns are going to have here. In particular we are going to be searching by device id an awful lot so to help speed those queries up and prevent duplicates we can tell Neo4j to treat that field a little differently.

In this case we aren't actually creating an index on this field directly but by placing a uniqueness constraint on the field an index is automatically created.

In [3]:
with driver.session() as session:
    session.run("CREATE CONSTRAINT device_id "
                "ON (device:Device) ASSERT device.id IS UNIQUE")

Now that we have created an index and a single device lets just make sure that we can read it back out of the database using a query.

In [4]:
def get_device_by_id(tx, deviceId):
    devices = tx.run("MATCH (device: Device) "
                         "WHERE device.id = $id "
                         "RETURN device", id=deviceId)
    return [ data[0] for data in devices ]

with driver.session() as session:
    result = session.read_transaction(get_device_by_id, "000000000000")
    dataFrame = DataFrame.from_records(result)
    display(dataFrame)

,id,name,type
0,000000000000,Device 1A,IntelPC


### Requirement - Find Device at a Given Location

Ok so we aren't exactly sticking to an order here but to a large extent the advantages of a graph database are related to it's modelling of relationships so I wanted to get to them sooner rather than later. For this section we are going to clear the database and add the sample data shown below.

In [5]:
with driver.session() as session:
    session.run("MATCH (a)-[b]->(c) DELETE b")
    session.run("MATCH (a) DELETE a")

devices = read_csv("./data/devices_stage_1.csv")
devices

,id,name,type
0,800000000000,Device 1A,Intel
1,800000000001,Device 1B,Intel
2,800000000002,RaspberryPi 1A,RaspberryPi
3,800000000003,RaspberryPi 2B,RaspberryPi
4,800000000004,Compact 1A,Apple
5,800000000005,Compact 2B,Apple
6,800000000006,Embedded 1,AMD
7,800000000007,Embedded 2,AMD
8,800000000008,Trainer 1,Bicycle
9,800000000009,Trainer 2,Bicycle


In [6]:
sites = read_csv("./data/sites_stage_1.csv")
sites

,id,name
0,site0001,Ostrich
1,site0002,Sandy Beach
2,site0003,London Primary
3,site0004,London Secondary
4,site0005,Mudhall
5,site0006,Hull


In [7]:
relationships = read_csv("./data/site_devices_stage_1.csv")
relationships

,site,device
0,site0001,800000000000
1,site0002,800000000001
2,site0003,800000000002
3,site0002,800000000003
4,site0002,800000000004
5,site0004,800000000005
6,site0002,800000000006
7,site0001,800000000007
8,site0005,800000000008
9,site0006,800000000009


In [8]:
def create_site(tx, site):
    result = tx.run("CREATE (a:Site) "
                    "SET a = $site "
                    "RETURN a", site=site)
    return result.single()

def relate_device_to_site(tx, relationship):
    result = tx.run("MATCH (a: Device), (b: Site) "
                    "WHERE a.id = $deviceId and b.id = $siteId "
                    "CREATE (b)-[r: HAS_DEVICE]->(a)"
                    "RETURN r", deviceId=relationship['device'], siteId=relationship['site'])
    return [ data[0] for data in result ]

with driver.session() as session:
    for device in devices.to_dict('records'):
        session.write_transaction(create_device, device)
    
    for site in sites.to_dict('records'):
        session.write_transaction(create_site, site)
        
    for relationship in relationships.to_dict('records'):
        session.write_transaction(relate_device_to_site, relationship)
    
    display(draw(session, { 'Device': 'name', 'Site': 'name'}, True))

Even after writing this query I'm still not sure I have the directionality of that relationship right but it still works. However if we go back to the requirement we want to be able to find all the devices for a particular site. Lets see what that query would look like if we wanted to find the devices on site0001.

In [9]:
def get_devices_by_site(tx, siteId):
    devices = tx.run("MATCH (site: Site)-[:HAS_DEVICE]->(device: Device) "
                         "WHERE site.id = $id "
                         "RETURN device", id=siteId)
    return [ data[0] for data in devices ]

with driver.session() as session:
    result = session.read_transaction(get_devices_by_site, "site0001")
    dataFrame = DataFrame.from_records(result)
    display(dataFrame)

,id,name,type
0,800000000007,Embedded 2,AMD
1,800000000000,Device 1A,Intel


I think it's worth taking a step back at this point and considering what the relationship we have just made. Unlike a relational database we have not had to alter the schema of either node (or table if you prefer). This means that relationships can be added and removed without easily as requirements change. Compared to a document store database we have not had to worry about denormalising any data or which pieces of data should go into what index for each query pattern.

Also although not standard across all graph databases the Neo4j query syntax of *(site: Site)-[:HAS_DEVICE]->(device: Device)* makes it obvious that we are searching for part of the graph where a site and a device are linked by a has device relationship.

To show how this model and continue to be expanded without having to make complicated migrations let move onto another of the requirements...

### Requirement - See All The Devices An Organisation Is Associated With

To meet this requirement we are going to have to introduce the concept of an organisation to our graph so lets get a small amount of sample data lined up for that first.

In [10]:
organisations = read_csv("./data/organisations_stage_1.csv")
organisations

,id,name
0,kahoot,Kahoot
1,cyberdyne,CyberDyne
2,coop,The Cooperative


In [11]:
organisation_relationships = read_csv("./data/organisation_sites_stage_1.csv")
organisation_relationships

,site,organisation
0,site0001,kahoot
1,site0002,cyberdyne
2,site0003,kahoot
3,site0004,cyberdyne
4,site0005,coop
5,site0006,kahoot


Having got some sample data the next step is to put the data into our graph. As you will see from the outpuit below suddenly our information is looking a lot more graph like!

In [12]:
def create_organisation(tx, organisation):
    result = tx.run("CREATE (a:Organisation) "
                    "SET a = $organisation "
                    "RETURN a", organisation=organisation)
    return result.single()

def relate_site_to_organisation(tx, relationship):
    result = tx.run("MATCH (a: Organisation), (b: Site) "
                    "WHERE a.id = $organisationId and b.id = $siteId "
                    "CREATE (a)-[r: HAS_SITE]->(b)"
                    "RETURN r", organisationId=relationship['organisation'], siteId=relationship['site'])
    return [ data[0] for data in result ]

with driver.session() as session:
    for organisation in organisations.to_dict('records'):
        session.write_transaction(create_organisation, organisation)
            
    for relationship in organisation_relationships.to_dict('records'):
        session.write_transaction(relate_site_to_organisation, relationship)
    
    display(draw(session, { 'Device': 'name', 'Site': 'name', 'Organisation': 'name' }, True))

Know that we have a graph with more than one degree of separation in it lets see what it is like to query it. First off lets just traverse the new has site relationship to find all the sites associated with the Kahoot organisation.

In [13]:
with driver.session() as session:
    result = session.run("MATCH (a: Organisation)-[:HAS_SITE]->(c) "
                "WHERE a.id = $organisationId "
                "RETURN c", organisationId='kahoot')
    dataFrame = DataFrame.from_records([ data[0] for data in result])
    display(dataFrame)

,id,name
0,site0006,Hull
1,site0003,London Primary
2,site0001,Ostrich


Now that we now that is working lets extend the match query to include two relationship. If you look at the code below you will see that the structure of the query is much the same as before. One thing you might notice is that there is simply a pair empty brackets where the site should be. That is becuase we do not want to extract the site information. In this case those empty brackets will match any node that has an incoming has site relationship and an outgoing has device relationship.

In [14]:
with driver.session() as session:
    result = session.run("MATCH (a: Organisation)-[:HAS_SITE]->()-[:HAS_DEVICE]->(c) "
                "WHERE a.id = $organisationId "
                "RETURN c", organisationId='kahoot')
    dataFrame = DataFrame.from_records([ data['c'] for data in result])
    display(dataFrame)

,id,name,type
0,800000000009,Trainer 2,Bicycle
1,800000000002,RaspberryPi 1A,RaspberryPi
2,800000000007,Embedded 2,AMD
3,800000000000,Device 1A,Intel


Whoops... we have forgotten that there are two types of organisation to site relationship that we need to manage. An organisation can own a site, manage it or both! We will need to change our model to reflect this.

There are two ways in which we could achieve this. We could either add a property to the relationship between the nodes to specify the type of relationship or have two different relationships. In this case it will probably be easier to have two different relationships so lets look at changing the current has site relationship to owns site and then adding a separate manages site relationship.

In [15]:
with driver.session() as session:
    session.run("MATCH (a: Organisation)-[rel:HAS_SITE]->(b: Site) "
                "MERGE (a)-[:OWNS_SITE]->(b) "
                "DELETE rel")
    display(draw(session, { 'Device': 'name', 'Site': 'name', 'Organisation': 'name' }, True))

Well that was easy! Having changed that relationship we must now load the data for the new relationship and insert it into the graph.

In [16]:
management_relationships = read_csv("./data/manage_sites_stage_1.csv")
management_relationships

,site,organisation
0,site0001,kahoot
1,site0002,coop
2,site0003,kahoot
3,site0004,coop
4,site0005,coop
5,site0006,kahoot


In [17]:
def relate_site_to_management_organisation(tx, relationship):
    result = tx.run("MATCH (a: Organisation), (b: Site) "
                    "WHERE a.id = $organisationId and b.id = $siteId "
                    "CREATE (a)-[r: MANAGES_SITE]->(b)"
                    "RETURN r", organisationId=relationship['organisation'], siteId=relationship['site'])
    return [ data[0] for data in result ]

with driver.session() as session:
    for relationship in management_relationships.to_dict('records'):
        session.write_transaction(relate_site_to_management_organisation, relationship)
    
    display(draw(session, { 'Device': 'name', 'Site': 'name', 'Organisation': 'name' }, True))

Our graph is definitely getting a little more complicated but before continuing lets make sure we can still find all the sites associated to a particular organisation as that was one of our requirements.

In [18]:
with driver.session() as session:
    result = session.run("MATCH (a: Organisation)-[:OWNS_SITE | MANAGES_SITE]->()-[:HAS_DEVICE]->(c) "
                "WHERE a.id = $organisationId "
                "RETURN c", organisationId='kahoot')
    dataFrame = DataFrame.from_records([ data['c'] for data in result])
    display(dataFrame)

,id,name,type
0,800000000009,Trainer 2,Bicycle
1,800000000002,RaspberryPi 1A,RaspberryPi
2,800000000007,Embedded 2,AMD
3,800000000000,Device 1A,Intel
4,800000000007,Embedded 2,AMD
5,800000000000,Device 1A,Intel
6,800000000009,Trainer 2,Bicycle
7,800000000002,RaspberryPi 1A,RaspberryPi


Ok... that's not right. If you look carefully at the results you can see that they are correct but they are duplicated. This is because ther are now two paths that we can traverse between the organisation and the devices. We need to fix this by asking for the unique results, distinct in Neo4j terms.

In [19]:
with driver.session() as session:
    result = session.run("MATCH (a: Organisation)-[:OWNS_SITE | MANAGES_SITE]->()-[:HAS_DEVICE]->(c) "
                "WHERE a.id = $organisationId "
                "RETURN DISTINCT c", organisationId='kahoot')
    dataFrame = DataFrame.from_records([ data['c'] for data in result])
    display(dataFrame)

,id,name,type
0,800000000009,Trainer 2,Bicycle
1,800000000002,RaspberryPi 1A,RaspberryPi
2,800000000007,Embedded 2,AMD
3,800000000000,Device 1A,Intel


### Requirement - Find Devices with a Particular Status

It probably hasn't escaped your notice that there isn't a status field on any of the devices. Well the cool thing is that the nodes are schemaless so I can just add one although there is a different approach we might consider. You can add additional labels to categorise nodes in different ways.

For our example we are going to label all the nodes owned by cyberdyne as ACTIVE. We can then run a query to find all the ACTIVE nodes in the graph. Note you could do this in one operation but I just want to prove the query by label works.

In [20]:
with driver.session() as session:
    session.run("MATCH (a: Organisation)-[:OWNS_SITE]->()-[:HAS_DEVICE]->(b) "
                "WHERE a.id = $organisationId "
                "SET b:ACTIVE "
                "RETURN b", organisationId='cyberdyne')
    
    result = session.run("MATCH (a: ACTIVE) "
                "RETURN a");
    
    dataFrame = DataFrame.from_records([ data['a'] for data in result])
    display(dataFrame)

,id,name,type
0,800000000001,Device 1B,Intel
1,800000000003,RaspberryPi 2B,RaspberryPi
2,800000000004,Compact 1A,Apple
3,800000000005,Compact 2B,Apple
4,800000000006,Embedded 1,AMD


So labels are good for when we just want to categorise things and be able to find them quickly. However sometimes you just want to search by a property and that ok too! In fact that's exactly what we are going to do for our last requirement.

### Requirement - Find Device with a Particular Type
Interestingly if you have been paying attention you might have noticed that we have already done searches where we have filtered by a property - specifically we did this when searching for organisations by name using the where clause. However for this search we are going to use a flightly different format.

In [21]:
with driver.session() as session:
    result = session.run("MATCH (a: Device{ type: 'Intel' }) "
                "RETURN a");
    
    dataFrame = DataFrame.from_records([ data['a'] for data in result])
    display(dataFrame)

,id,name,type
0,800000000000,Device 1A,Intel
1,800000000001,Device 1B,Intel


As you can see from the query we ask Neo4j to find all the nodes with the label Device and a type property with a value of Intel... simple!

## Summary
I hope this quick overview of the capabilities of a graph database has piqued your interest. It does seem that for some models it provides a much more natural expression and it is something we should be considering whenever we are looking at creating new data stores.

I hope to add more examples to this document over time so if there are other requirements you want to suggest then please get in touch.

That only leaves one more thing to do, close the driver and free the system resources!

In [22]:
driver.close()